# <span style="font-width:bold; font-size: 3rem; color:#1EB182;">**Hopsworks Feature Store** </span> <span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Model training & UI Exploration</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/electricity/4_model_training_and_registration.ipynb)


## 🗒️ This notebook is divided into 3 main sections:
1. **Loading the training data**
2. **Train the model**
3. **Register model to Hopsworks model registry**.

![tutorial-flow](https://github.com/logicalclocks/hopsworks-tutorials/blob/master/images/03_model.png?raw=1)

### <span style="color:#ff5f27;">📝 Importing Libraries</span>

In [ ]:
!pip install -U hopsworks --quiet

In [ ]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression

from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings('ignore')

---

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

---

## <span style="color:#ff5f27;">🪝 Feature View and Training Dataset Retrieval</span>

In [ ]:
feature_view = fs.get_feature_view(
    name = 'citibike_feature_view',
    version = 1
)

In [ ]:
X_train, _ = feature_view.get_training_data(1)
X_test, _ = feature_view.get_training_data(2)

In [ ]:
X_train.head(3)

In [ ]:
X_train = X_train.drop(columns=["station_id"])
X_test = X_test.drop(columns=["station_id"])

In [ ]:
X_train.shape, X_test.shape

In [ ]:
X_train = X_train.set_index("date")
X_test = X_test.set_index("date")

In [ ]:
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

In [ ]:
y_train, y_test = X_train.pop("users_count_next_day"), X_test.pop("users_count_next_day")

---

## <span style="color:#ff5f27;">🧬 Modeling</span>

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
mlp = MLPRegressor(random_state=1, max_iter=100, activation="logistic", solver="adam",
                    learning_rate_init=0.01).fit(X_train, y_train)
pred_mlp = mlp.predict(X_test)

r2_mlp = r2_score(pred_mlp, y_test.values)
print("R2 score for SVR model:", r2_mlp)

### Remember, the purpose of this tutorial is to show your Hopsworks Feature Store functionality, so the perfect model is not our goal.

---

## <span style='color:#ff5f27'>🗄 Model Registry</span>

One of the features in Hopsworks is the model registry. This is where you can store different versions of models and compare their performance. Models from the registry can then be served as API endpoints.

In [ ]:
mr = project.get_model_registry()

### <span style="color:#ff5f27;">⚙️ Model Schema</span>

The model needs to be set up with a [Model Schema](https://docs.hopsworks.ai/machine-learning-api/latest/generated/model_schema/), which describes the inputs and outputs for a model.

A Model Schema can be automatically generated from training examples, as shown below.

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

model_schema.to_dict()

In [ ]:
import joblib

joblib.dump(mlp, 'model.pkl')

In [ ]:
model = mr.sklearn.create_model(
    name="citibike_mlp_model",
    version=1,
    metrics={"r2_score": r2_mlp},
    description="MLPRegressor. Citibike Project.",
    input_example=X_train.sample(),
    model_schema=model_schema
)

model.save('model.pkl')